**《深度学习之 PyTorch 实战》**

讲师作者：[土豆老师](https://iphysresearch.github.io)

# 循环神经网络

## 门控循环单元

>(Restart your kernel here)

上一讲介绍了循环神经网络中的梯度计算方法。我们发现，当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。虽然裁剪梯度可以应对梯度爆炸，但无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中较难捕捉时间序列中时间步距离较大的依赖关系。让我们简单思考一下这种梯度异常在实践中的意义：

- 我们可能会遇到这样一种情况——早期观测值对预测所有未来观测值具有非常重要的意义。考虑一个极端情况，其中第一个观测值包含一个校验和，目标是在序列的末尾辨别校验和是否正确。在这种情况下，第一个标记的影响至关重要。我们想有一些机制能够在一个记忆细胞里存储重要的早期信息。如果没有这样的机制，我们将不得不给这个观测值指定一个非常大的梯度，因为它会影响所有后续的观测值。

- 我们可能会遇到这样的情况——一些标记没有相关的观测值。例如，在解析网页时，可能有一些辅助HTML代码与评估网页上传达的情绪无关。我们希望有一些机制来跳过隐状态表示中的此类标记。

- 我们可能会遇到这样的情况——序列的各个部分之间存在逻辑中断。例如，书的章节之间可能会有一个过渡，或者证券的熊市和牛市之间可能会有一个过渡。在这种情况下，最好有一种方法来重置我们的内部状态表示。


**门控循环神经网络**（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。它通过可以学习的门来控制信息的流动。其中，**门控循环单元**（gated recurrent unit，GRU）是一种常用的门控循环神经网络。

### 门控循环单元

下面将介绍门控循环单元的设计。它引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式。

#### 重置门和更新门

如下图所示，门控循环单元中的重置门和更新门的输入均为当前时间步输入 $\boldsymbol{X}_{t}$ 与上一时间步隐藏状态 $\boldsymbol{H}_{t-1}$ ，输出由激活函数为 sigmoid 函数的全连接层计算得到。

![](https://i.loli.net/2021/05/21/UD8wN9hRQoqlngX.png)

具体来说，假设隐藏单元个数为 $h$, 给定时间步 $t$ 的小批量输入 $\boldsymbol{X}_{t} \in \mathbb{R}^{n \times d} \quad$ (样本数为 $n$, 输入个数为 $d)$ 和上一时 间步隐藏状态 $\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。 重置门 $\boldsymbol{R}_{t} \in \mathbb{R}^{n \times h}$ 和更新门 $\boldsymbol{Z}_{t} \in \mathbb{R}^{n \times h}$ 的计算如下:

$$\begin{aligned} \boldsymbol{R}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x r}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h r}+\boldsymbol{b}_{r}\right) \\ \boldsymbol{Z}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x z}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h z}+\boldsymbol{b}_{z}\right) \end{aligned}$$

其中 $\boldsymbol{W}_{x r}, \boldsymbol{W}_{x z} \in \mathbb{R}^{d \times h}$ 和 $\boldsymbol{W}_{h r}, \boldsymbol{W}_{h z} \in \mathbb{R}^{h \times h}$ 是权重参数, $\boldsymbol{b}_{r}, \boldsymbol{b}_{z} \in \mathbb{R}^{1 \times h}$ 是偏差参数。「多层感知机」节中介绍过，sigmoid 函数可以将元素的值变换到0和1之间。因此，重置门 $\boldsymbol{R}_{t}$ 和更新门 $\boldsymbol{Z}_{t}$ 中每个元素的值域者 是 $[0,1]$ 。


#### 候选隐藏状态

接下来，门控循环单元将计算候选隐藏状态来辅助稍后的隐藏状态计算。如下图所示，我们将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法 (符号为 $\odot$）。如果重置门中元素值接近 0, 那么意味着重置对应隐藏状态元素为 0, 即丢弃上一时间步的隐藏状态。如果元素值接近1，那么表示保留上一时间步的隐藏状态。然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数tanh的全连接层计算出候选隐藏状态，其所有元素的值域为 $[-1,1]$ 。

![](https://i.loli.net/2021/05/21/rt9ERH7G2BTISng.png)

具体来说, 时间步 $t$ 的候选隐藏状态 $\tilde{\boldsymbol{H}}_{t} \in \mathbb{R}^{n \times h}$ 的计算为

$$
\tilde{\boldsymbol{H}}_{t}=\tanh \left(\boldsymbol{X}_{t} \boldsymbol{W}_{x h}+\left(\boldsymbol{R}_{t} \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{h h}+\boldsymbol{b}_{h}\right)
$$

其中 $\boldsymbol{W}_{x h} \in \mathbb{R}^{d \times h}$ 和 $\boldsymbol{W}_{h h} \in \mathbb{R}^{h \times h}$ 是权重参数, $\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h} $ 是偏差参数。从上面这个公式可以看出, 重置门控
制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。而上一时间步的隐藏状态可能包含了时间序列截
至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。

#### 隐藏状态

最后, 时间步 $t$ 的隐藏状态 $\boldsymbol{H}_{t} \in \mathbb{R}^{n \times h}$ 的计算使用当前时间步的更新门 $\boldsymbol{Z}_{t}$ 来对上一时间步的隐藏状态 $\boldsymbol{H}_{t-1}$ 和业前时间步的候选隐藏状态 $\tilde{\boldsymbol{H}}_{t}$ 做组合: 

$$
\boldsymbol{H}_{t}=\boldsymbol{Z}_{t} \odot \boldsymbol{H}_{t-1}+\left(1-\boldsymbol{Z}_{t}\right) \odot \tilde{\boldsymbol{H}}_{t}
$$

![](https://i.loli.net/2021/05/21/PXlcxHrFCwaknZT.png)

值得注意的是，更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，如上图所示。假设更新门在时间步 $t^{\prime}$ 到 $t\left(t^{\prime}<t\right)$ 之间一直近似 1。那么，在时间步 $t^{\prime}$ 到t之间的输入信息几乎没有流入时间步 $t$ 的隐 藏状态 $\boldsymbol{H}_{t}$ 。实际上，这可以看作是较早时刻的隐藏状态 $\boldsymbol{H}_{t^{\prime}-1}$ 一直通过时间保存并传递至当前时间步 $t_{\circ}$ 这个设计可以应对循环神经网络中的梯度衰减问题, 并更好地捕捉时间序列中时间步距离较大的依赖关系。

我们对门控循环单元的设计稍作总结:

- 重置门有助于捕捉时间序列里短期的依赖关系;
- 更新门有助于捕捉时间序列里长期的依赖关系。

### 读取数据集

为了实现并展示门控循环单元，下面依然使用周杰伦歌词数据集来训练模型作词。这里除门控循环单元以外的实现已在上一节「循环神经网络」中介绍过。以下为读取数据集部分。

In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import dl4wm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = dl4wm.load_data_jay_lyrics()

### 从零开始实现

####  初始化模型参数

下面的代码对模型参数进行初始化。超参数 `num_hiddens` 定义了隐藏单元的个数。

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cpu


#### 定义模型

下面的代码定义隐藏状态初始化函数 `init_gru_state`。同「循环神经网络的从零开始实现」节中定义的 `init_rnn_state` 函数一样，它返回由一个形状为(批量大小, 隐藏单元个数)的值为 0 的 `Tensor` 组成的元组。

In [3]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )


下面根据门控循环单元的计算表达式定义模型。

In [4]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

#### 训练模型并创作歌词

我们在训练模型时只使用相邻采样。设置好超参数后，我们将训练模型并根据前缀“分开”和“不分开”分别创作长度为50个字符的一段歌词。

In [5]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

我们每过40个迭代周期便根据当前训练的模型创作一段歌词。

In [6]:
dl4wm.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)


epoch 40, perplexity 150.034250, time 1.47 sec
 - 分开 我想你的让我 你不你 我不不 我想你 我不要 我不不 我想你 我不要 我不不 我想你 我不要 我不
 - 不分开 我想你的让我 你不你 我不不 我想你 我不要 我不不 我想你 我不要 我不不 我想你 我不要 我不
epoch 80, perplexity 31.731070, time 1.41 sec
 - 分开 我想要这样 我有一定直 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开  没有你在我有多难难熬                                      
epoch 120, perplexity 5.904515, time 1.50 sec
 - 分开 一直心酒 你在那空 在小村外的溪边 默默等待 是谁的脚娘 银制茶壶 全始都外 装果在空凉义 白北的
 - 不分开我 爸不知觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 
epoch 160, perplexity 1.749853, time 1.46 sec
 - 分开 一直心酒 你来的事息 还真安动的狗剩河清晰 深已说当 娘时 一壶 再著再人 每偷在空演 我也就很走
 - 不分开  没有回烦 我有多烦恼  没有你烦我有多烦恼多烦恼  穿过云层 我试著努力向你奔跑 爱才送到 你却


### 简洁实现

在 PyTorch 中我们直接调用 `nn` 模块中的 `GRU` 类即可。

In [7]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = dl4wm.RNNModel(gru_layer, vocab_size).to(device)
dl4wm.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 1.014931, time 1.22 sec
 - 分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
 - 不分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
epoch 80, perplexity 1.016674, time 1.25 sec
 - 分开的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后在这里 不限日期 然后将过去 慢慢温习 让我爱
 - 不分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
epoch 120, perplexity 1.010166, time 1.27 sec
 - 分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
 - 不分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
epoch 160, perplexity 1.011896, time 1.27 sec
 - 分开的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后在这里 不限日期 然后将过去 慢慢温习 让我爱
 - 不分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让


### 小结

- 门控循环神经网络可以更好地捕捉时间序列中时间步距离较大的依赖关系。
- 门控循环单元引入了门的概念，从而修改了循环神经网络中隐藏状态的计算方式。它包括重置门、更新门、候选隐藏状态和隐藏状态。
- 重置门有助于捕捉时间序列里短期的依赖关系。
- 更新门有助于捕捉时间序列里长期的依赖关系。
- 重置门打开时，门控循环单元包含基本循环神经网络；更新门打开时，门控循环单元可以跳过子序列。

## 长短期记忆（LSTM）

>(Restart your kernel here)

本节将介绍另一种常用的门控循环神经网络：**长短期记忆**（long short-term memory，LSTM）。它比门控循环单元的结构稍微复杂一点。

可以说，长短期记忆网络的设计灵感来自于计算机的逻辑门。长短期记忆网络引入了存储单元（memory cell），或简称为单元（cell）。（一些文献认为存储单元是隐藏状态的一种特殊类型）。它们与隐藏状态具有相同的形状，被设计为记录附加信息。为了控制存储单元，我们需要许多门。如，我们需要一个门来从单元中读出条目。我们将其称为输出门（output gate）。 另外，需要一个门来决定何时将数据读入单元。我们将其称为输入门（input gate）。最后，我们需要一种机制来重置单元的内容，由遗忘门（forget gate）来管理。这种设计的动机与门控循环单元相同，即能够通过专用机制决定什么时候记忆或忽略隐藏状态中的输入。让我们看看这在实践中是如何运作的。

### 长短期记忆

LSTM 中引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞（某些文献把记忆细胞当成一种特殊的隐藏状态），从而记录额外的信息。

#### 输入门、遗忘门和输出门

与门控循环单元中的重置门和更新门一样，如下图所示，长短期记忆的门的输入均为当前时间步输入 $\boldsymbol{X}_{t}$ 与上一时
间步隐藏状态 $\boldsymbol{H}_{t-1}$, 输出由激活函数为 $\operatorname{sigmoid}$ 函数的全连接层计算得到。如此一来，这3个门元素的值域均为 $[0,1]$。

![](https://i.loli.net/2021/05/21/4N1GUXIyRVoYrju.png)

具体来说，假设隐藏单元个数为 $h$, 给定时间步 $t$ 的小批量输入 $\boldsymbol{X}_{t} \in \mathbb{R}^{n \times d}$ (样本数为 $n$, 输入个数为 $d$) 和上一时 间步隐藏状态 $\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h} $。 时间步 $t$ 的输入门 $\boldsymbol{I}_{t} \in \mathbb{R}^{n \times h}$、 遗忘门 $\boldsymbol{F}_{t} \in \mathbb{R}^{n \times h}$ 和输出门 $\boldsymbol{O}_{t} \in \mathbb{R}^{n \times h}$ 分别计算
如下:

$$
\begin{aligned}
\boldsymbol{I}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x i}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h i}+\boldsymbol{b}_{i}\right) \\
\boldsymbol{F}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x f}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h f}+\boldsymbol{b}_{f}\right) \\
\boldsymbol{O}_{t} &=\sigma\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x o}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h o}+\boldsymbol{b}_{o}\right)
\end{aligned}
$$

其中的 $\boldsymbol{W}_{x i}, \boldsymbol{W}_{x f}, \boldsymbol{W}_{x o} \in \mathbb{R}^{d \times h}$ 和 $\boldsymbol{W}_{h i}, \boldsymbol{W}_{h f}, \boldsymbol{W}_{h o} \in \mathbb{R}^{h \times h}$ 是权重参数, $\boldsymbol{b}_{i}, \boldsymbol{b}_{f}, \boldsymbol{b}_{o} \in \mathbb{R}^{1 \times h}$ 是偏差参数

#### 候选记忆细胞

接下来，长短期记忆需要计算候选记忆细胞 $\tilde{C}_{t \circ}$ 它的计算与上面介绍的3个门类似, 但使用了值域在 $[-1,1]$ 的  tanh 函数作为激活函数，如下图所示。

![](https://i.loli.net/2021/05/21/amsSRetz4GjlAhn.png)

具体来说，时间步 $t$ 的候选记忆细胞 $\tilde{\boldsymbol{C}}_{t} \in \mathbb{R}^{n \times h}$ 的计算为

$$
\tilde{\boldsymbol{C}}_{t}=\tanh \left(\boldsymbol{X}_{t} \boldsymbol{W}_{x c}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h c}+\boldsymbol{b}_{c}\right)
$$

其中 $\boldsymbol{W}_{x c} \in \mathbb{R}^{d \times h}$ 和 $\boldsymbol{W}_{h c} \in \mathbb{R}^{h \times h}$ 是权重参数, $\boldsymbol{b}_{c} \in \mathbb{R}^{1 \times h}$ 是偏差参数。

#### 记忆细胞

我们可以通过元素值域在 $[0,1]$ 的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按 元素乘法 (符号为 $\odot$ ) 来实现的。当前时间步记忆细胞 $\boldsymbol{C}_{t} \in \mathbb{R}^{n \times h}$ 的计算组合了上一时间步记忆细胞和当前时间 步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动:

$$
\boldsymbol{C}_{t}=\boldsymbol{F}_{t} \odot \boldsymbol{C}_{t-1}+\boldsymbol{I}_{t} \odot \tilde{\boldsymbol{C}}_{t}
$$

如图下图所示，遗忘门控制上一时间步的记忆细胞 $\boldsymbol{C}_{t-1}$ 中的信息是否传递到当前时间步，而输入门则控制当前时间 步的输入 $\boldsymbol{X}_{t}$ 通过候选记忆细胞 $\tilde{\boldsymbol{C}}_{t}$ 如何流入当前时间步的记忆细胞。如果遗忘门一直近似 1 且输入门一直近似 0, 过去的记忆细胞将一直通过时间保存并传递至当前时间步。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

![](https://i.loli.net/2021/05/21/krsJedYybom4A8x.png)


#### 隐藏状态

有了记忆细胞以后，接下来我们还可以通过输出门来控制从记忆细胞到隐藏状态 $\boldsymbol{H}_{t} \in \mathbb{R}^{n \times h}$ 的信息的流动:

$$
\boldsymbol{H}_{t}=\boldsymbol{O}_{t} \odot \tanh \left(\boldsymbol{C}_{t}\right)
$$

这里的 tanh 函数确保隐藏状态元素值在 -1 到 1 之间。需要注意的是，当输出门近似 1 时，记忆细胞信息将传递到隐藏状态供输出层使用; 当输出门近似 0 时，记忆细胞信息只自己保留。下图展示了长短期记忆中隐藏状态的计算。

![](https://i.loli.net/2021/05/21/nEyCaxowRXLuOIb.png)


###  读取数据集

下面我们开始实现并展示长短期记忆。和前几节中的实验一样，这里依然使用周杰伦歌词数据集来训练模型作词。

In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import dl4wm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = dl4wm.load_data_jay_lyrics()

### 从零开始实现

我们先介绍如何从零开始实现长短期记忆。

#### 初始化模型参数

下面的代码对模型参数进行初始化。超参数 `num_hiddens` 定义了隐藏单元的个数。


In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])


will use cpu


### 定义模型

在初始化函数中，长短期记忆的隐藏状态需要返回额外的形状为(批量大小, 隐藏单元个数)的值为0的记忆细胞。

In [3]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))


下面根据长短期记忆的计算表达式定义模型。需要注意的是，只有隐藏状态会传递到输出层，而记忆细胞不参与输出层的计算。

In [4]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)


#### 训练模型并创作歌词

同上一节一样，我们在训练模型时只使用相邻采样。设置好超参数后，我们将训练模型并根据前缀“分开”和“不分开”分别创作长度为50个字符的一段歌词。

In [5]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']


我们每过40个迭代周期便根据当前训练的模型创作一段歌词。

In [6]:
dl4wm.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)


epoch 40, perplexity 214.437860, time 1.59 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 66.426345, time 1.63 sec
 - 分开 我想你这你 我不要 我不了我 我不了我 我不了我 我不了我 我不了我 我不了我 我不了我 我不了我
 - 不分开 我想你这你 我不要 我不了我 我不了我 我不了我 我不了我 我不了我 我不了我 我不了我 我不了我
epoch 120, perplexity 16.625859, time 1.59 sec
 - 分开 我想你这生很 一天后 一直我 一直怎么 在有了空 在在风空 在在风空 在不了空 在不了空 在不了空
 - 不分开 我爱你这生堡 一天后 一直我 一直我 一直怎么 在有了空 在在了空 在在风空 在不了空 在不了空 
epoch 160, perplexity 4.066085, time 1.64 sec
 - 分开 你已我 你常么 是手怎么不留 有直是双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈
 - 不分开 想要你你已我 每你球 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的


### 简洁实现

在 Pytorch 中，我们可以直接调用 `rnn` 模块中的 `LSTM` 类。

In [7]:
lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = dl4wm.RNNModel(lstm_layer, vocab_size)
dl4wm.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 1.019654, time 1.29 sec
 - 分开 它所拥有的只剩下回忆 相爱还有别离 像无法被安排的雨 随时准备来袭 我怀念起国小的课桌椅 怀念著用
 - 不分开 它所拥有的只剩下回忆 相爱还有别离 像无法被安排的雨 随时准备来袭 我怀念起国小的课桌椅 怀念著用
epoch 80, perplexity 1.016015, time 1.28 sec
 - 分开 它所拥有的只剩下回忆 相爱还有别离 像无法被安排的雨 随时准备来袭 我怀念起国小的课桌椅 怀念著用
 - 不分开 我满了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我
epoch 120, perplexity 1.009871, time 1.28 sec
 - 分开 它所拥有的只剩下回忆 相爱还有别离 像无法被安排的雨 随时准备来袭 我怀念起国小的课桌椅 用铅笔写
 - 不分开 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 我不能 爱情走的
epoch 160, perplexity 1.010431, time 1.27 sec
 - 分开始玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 不想太多 我想一定是我听错弄错搞
 - 不分开 我爱你看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害


### 小结

- 长短期记忆的隐藏层输出包括隐藏状态和记忆细胞。只有隐藏状态会传递到输出层。
- 长短期记忆的输入门、遗忘门和输出门可以控制信息的流动。
- 长短期记忆可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

## 深层循环神经网络

>(Restart your kernel here)

到目前为止，我们只讨论了一个单向隐藏层的RNN。其中，隐变量和观测值如何相互作用的具体函数形式是相当任意的。只要我们有足够的灵活性来建模不同类型的交互，这就不是一个大问题。然而，对于一个单隐藏层来说，这可能是相当具有挑战性的。在线性模型的情况下，我们通过添加更多的层来解决这个问题。在循环神经网络中，这有点棘手。因为我们首先需要决定如何以及在哪里添加额外的非线性函数。

事实上，我们可以将多层循环神经网络堆叠在一起。通过几个简单的层的组合，产生了一个灵活的机制。特别是，数据可能与堆叠的不同层级有关。例如，我们可能希望保持有关金融市场状况（熊市或牛市）的高级数据可用，而在较低级别，我们只记录较短期的时间动态。

除了以上所有的抽象讨论之外，通过下图可能更容易理解我们感兴趣的模型。它描述了一个具有 $L$ 个隐藏层的深层循环神经网络。每个隐藏状态都连续传递到当前层的下一个时间步和下一层的当前时间步。

![](https://i.loli.net/2021/05/21/EKj2LFwvAqxCIWN.png)


###  函数依赖关系

我们可以在上图中描述的 $L$ 个隐藏层的深层结构中的函数依赖关系形式化。我们下面的讨论主要集中在经典循环神经网络模型上，但它也适用于其他序列模型。

假设我们在时间步 $t$ 有一个小批量输入 $\mathbf{X}_{t} \in \mathbb{R}^{n \times d}$ (样本数： $n$, 每个样本中的输入数：d）。同时, 将 $l^{\text {th }}$ 隐藏层 $(l=1, \ldots, L)$ 的隐藏状态设为 $\mathbf{H}_{t}^{(l)} \in \mathbb{R}^{n \times h}$ (隐藏单元数： $h$), 输出层变量设为 $\mathbf{O}_{t} \in \mathbb{R}^{n \times q}($ 输出数： $q)$ 。设置 $\mathbf{H}_{t}^{(0)}=\mathbf{X}_{t}$, 使用激活函数 $\phi_{l}$ 的第l个隐藏层的隐藏状态表示如
下:

$$\mathbf{H}_{t}^{(l)}=\phi_{l}\left(\mathbf{H}_{t}^{(l-1)} \mathbf{W}_{x h}^{(l)}+\mathbf{H}_{t-1}^{(l)} \mathbf{W}_{h h}^{(l)}+\mathbf{b}_{h}^{(l)}\right)$$

其中, 权重 $\mathbf{W}_{x h}^{(l)} \in \mathbb{R}^{h \times h}$ 和 $\mathbf{W}_{h h}^{(l)} \in \mathbb{R}^{h \times h}$ 以及偏置 $\mathbf{b}_{h}^{(l)} \in \mathbb{R}^{1 \times h}$ 是第l个隐藏层的模型参数。
最后，输出层的计算仅基于最终第l个隐藏层的隐藏状态:

$$
\mathbf{O}_{t}=\mathbf{H}_{t}^{(L)} \mathbf{W}_{h q}+\mathbf{b}_{q},
$$

其中, 权重 $\mathbf{W}_{h q} \in \mathbb{R}^{h \times q}$ 和偏置b $_{q} \in \mathbb{R}^{1 \times q}$ 是输出层的模型参数。

与多层感知机一样，隐藏层的数目 $L$ 和隐藏单元的数目 $h$ 是超参数。换句话说，它们可以由我们调整或指定。另外，用门控循环单元或长短期记忆网络的隐藏状态计算代替上上式子的隐藏状态计算，可以很容易地得深层门控循环神经网络。

### 简洁实现

幸运的是，实现多层循环神经网络所需的许多细节在高级API中都是现成的。为了简单起见，我们仅说明使用此类内置函数的实现。让我们以长短期记忆网络模型为例。该代码与我们之前在「长短期记忆（LSTM）」节中使用的代码非常相似。实际上，唯一的区别是我们显式地指定了层的数量，而不是单个层的默认值。像往常一样，我们从加载数据集开始。

In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import dl4wm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = dl4wm.load_data_jay_lyrics()

In [2]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

选择超参数等结构决策与上一节的决策非常相似。我们选择相同数量的输入和输出，因为我们有不同的标记，即 `vocab_size`。隐藏单元的数量仍然是256。唯一的区别是，我们现在通过指定 `num_layers` 的值来指定隐藏层数。

In [3]:
num_inputs, num_hiddens, num_outputs, num_layers = vocab_size, 256, vocab_size, 2
print('will use', device)

will use cpu


In [4]:
lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens, num_layers=num_layers)
model = dl4wm.RNNModel(lstm_layer, vocab_size)
dl4wm.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 10.591622, time 2.53 sec
 - 分开 我想要再这样的让我疯狂的可爱女人 一颗两颗三颗四颗 连成线背著背默默许下默默默心愿的可呼在远著现 
 - 不分开 一场悲剧 我想你你你妈 我不想 我不多 你 我不着 我要去 没去 想你 我要开 一颗两颗三颗四颗 
epoch 80, perplexity 1.408830, time 2.51 sec
 - 分开 有话 这样牵着你 没有你烦我有多烦恼多难熬  穿过云层 我试著努力向你奔跑 却才已一场盯人防守 整
 - 不分开不能听 捏成你的微笑每天都能看到  我知道这里很美但家乡的你更美原来我为一场悲剧 我想我这辈子注定一
epoch 120, perplexity 1.020499, time 2.53 sec
 - 分开 我不要再想 我不 我不 我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我
 - 不分开不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发抖 快给我
epoch 160, perplexity 1.015708, time 2.48 sec
 - 分开 我说我说 话就儿有不反鸦昏头的响尾蛇 无力的躺在干枯的河 在等待雨季来临变沼泽 灰狼啃食著水鹿的骨
 - 不分开 想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 


因为现在我们用长短期记忆网络模型实例化了两个层，这个相当复杂的结构大大降低了训练速度。

### 小结

- 在深层循环神经网络中，隐藏状态信息被传递到当前层的下一时间步和下一层的当前时间步。

- 有许多不同风格的深层循环神经网络，如长短期记忆网络、门控循环单元、或经典循环神经网络。这些模型在深度学习框架的高级API中都有涵盖。

- 总体而言，深层循环神经网络需要大量的工作(如学习率和修剪)来确保适当的收敛，模型的初始化也需要谨慎。

## 双向循环神经网络

>(Restart your kernel here)

在序列学习中，我们以往假设的目标是：到目前为止，在给定所观测的情况下对下一个输出进行建模。例如，在时间序列的上下文中或在语言模型的上下文中。虽然这是一个典型的情况，但这并不是我们可能遇到的唯一情况。为了说明这个问题，考虑以下三个在文本序列中填空的任务：

- 我 ___。
- 我 ___ 饿了。
- 我 ___ 饿了，我可以吃半头猪。

根据可获得的信息量，我们可以用不同的词填空，如“很高兴”（“happy”）、“不”（“not”）和“非常”（“very”）。很明显，短语的结尾（如果有的话）传达了重要信息。这些信息关乎到选择哪个词来填空。不能利用这一点的序列模型将在相关任务上表现不佳。例如，如果要做好命名实体识别（例如，识别“Green”指的是“格林先生”还是绿色），更长的范围上下文同样重要。

### 双向模型

我们从最后一个标记开始从后向前运行循环神经网络，而不是只在前向模式下从第一个标记开始运行循环神经网络。 双向循环神经网络添加了反向传递信息的隐藏层，以更灵活地处理此类信息。 下图具有单个隐藏层的双向循环神经网络的结构。

![](https://i.loli.net/2021/05/21/DEimcQ5MnfJyozS.png)



#### 定义

双向循环神经网络由 [Schuster & Paliwal, 1997] 提出。有关各种结构的详细讨论，请参阅 Graves.Schmidhuber.2005。让我们看看这样一个网络的具体情况。

对于任意时间步 $t$, 给定一个小批量输入 $\mathbf{X}_{t} \in \mathbb{R}^{n \times d}$（样本数：$n$, 每个示例中的输入数： $d$ ), 并且 使隐藏层激活函数为 $\phi_{\circ}$ 在双向结构中，我们设该时间步的前向和反向隐藏状态分别为 $\overrightarrow{\mathbf{H}}_{t} \in \mathbb{R}^{n \times h}$ 和 $\overleftarrow{\mathbf{H}}_t \in \mathbb{R}^{n \times h}$, 其中 $h$ 是隐藏单元的数目。前向和反向隐藏状态更新如下:

$$
\begin{aligned}
\overrightarrow{\mathbf{H}}_{t} &=\phi\left(\mathbf{X}_{t} \mathbf{W}_{x h}^{(f)}+\overrightarrow{\mathbf{H}}_{t-1} \mathbf{W}_{h h}^{(f)}+\mathbf{b}_{h}^{(f)}\right) \\
\overline{\mathbf{H}}_{t} &=\phi\left(\mathbf{X}_{t} \mathbf{W}_{x h}^{(b)}+\check{\mathbf{H}}_{t+1} \mathbf{W}_{h h}^{(b)}+\mathbf{b}_{h}^{(b)}\right)
\end{aligned}
$$

其中, 权重 $\mathbf{W}_{x h}^{(f)} \in \mathbb{R}^{d \times h}, \mathbf{W}_{h h}^{(f)} \in \mathbb{R}^{h \times h}, \mathbf{W}_{x h}^{(b)} \in \mathbb{R}^{d \times h}$, and $\mathbf{W}_{h h}^{(b)} \in \mathbb{R}^{h \times h}$ 和偏置 $\mathbf{b}_{h}^{(f)} \in \mathbb{R}^{1 \times h}$ and $\mathbf{b}_{h}^{(b)} \in \mathbb{R}^{1 \times h}$ 都是模型参数。

接下来，我们连结前向和反向隐藏状态 $\overrightarrow{\mathbf{H}}_{t}$ 和 $\overleftarrow{\mathbf{H}}_t$ 以获得要送入输出层的隐藏状态 $\mathbf{H}_{t} \in \mathbb{R}^{n \times 2 h}$ 在具 有多个隐藏层的深层双向循环神经网络中，该信息作为输入传递到下一个双向层。最后，输出层计算输出 $\mathbf{O}_{t} \in \mathbb{R}^{n \times q}$ (输出数： $q$):

$$
\mathbf{O}_{t}=\mathbf{H}_{t} \mathbf{W}_{h q}+\mathbf{b}_{q}
$$

这里，权重矩阵 $\mathbf{W}_{h q} \in \mathbb{R}^{2 h \times q}$ 和偏置 $\mathbf{b}_{q} \in \mathbb{R}^{1 \times q}$ 是输出层的模型参数。实际上，这两个方向可以有不同数量的隐藏单元。

#### 计算成本及其应用

双向循环神经网络的一个关键特性是，使用来自序列两端的信息来估计输出。也就是说，我们使用来自未来和过去观测的信息来预测当前的观测。在预测下一个标记的情况下，这并不是我们想要的。毕竟，在预测下一个标记时，我们无法知道下一个标记。因此，如果我们天真地使用双向循环神经网络，我们将不会得到很好的准确性：在训练期间，我们利用了过去和未来的数据来估计现在。而在测试期间，我们只有过去的数据，因此准确性较差。我们将在下面的实验中说明这一点。

另一个严重问题是，双向循环神经网络非常慢。其主要原因是前向传播需要在双向层中进行前向和后向递归，并且反向传播依赖于前向传播的结果。因此，梯度将有一个非常长的依赖链。

实际上，双向层的使用非常少，并且应用于部分场合。例如，填充缺失的单词、标记注释（例如，用于命名实体识别）以及作为序列处理工作流中的一个步骤对序列进行编码（例如，用于机器翻译）。

```python
bidirectional=True
```

### 小结

- 在双向循环神经网络中，每个时间步的隐藏状态由当前时间步前后信息同时决定。

- 双向循环神经网络与概率图形模型中的“前向-后向”算法有着惊人的相似性。

- 双向循环神经网络主要用于序列编码和给定双向上下文的观测估计。

- 由于梯度链更长，双向循环神经网络的训练成本非常高。